In [1]:
import pickle
import numpy as np
import pandas as pd
import torch

In [2]:
with open('rating_dict', 'rb') as f:
    rating_dict = pickle.load(f)

In [3]:
CF_user, CF_recom, CF_groundTrue = rating_dict['users_list'], rating_dict['rating_list'], rating_dict['groundTrue_list']

In [4]:
CBF_user = pd.read_csv('../user_list.csv')

In [5]:
CBF_user

Unnamed: 0                       0
0               0  mh_-eMZ6K5RLWhZyISBhwA
1               1  ejFxLGqQcWNLdNByJlIhnQ
2               2  f7xa0p_1V9lx53iIGN5Sug
3               3  qdneUwCll9ADHOp7i3wq3Q
4               4  QD9xhB-261YIQIFI5sRBtQ
...           ...                     ...
25503       25503  Dxp82KxFcmisgeCxHomfVg
25504       25504  wGqF01YaUYyF0aK8R8H9Lw
25505       25505  roGoHGWTkYrerml4wr6Wmg
25506       25506  P_bWyqJNstfYrFSLQjGrrQ
25507       25507  pwJJW342YyjEyq1Mxc9wXw

[25508 rows x 2 columns]

In [6]:
CBF_recom = torch.load('../cb_recom.pt')

In [7]:
CBF_recom[0]

tensor([18, 10,  4, 16, 12,  8, 19,  1, 11,  5, 15, 13,  7, 17,  3,  9,  0,  2,
         6, 14])

In [8]:
CBF_user = CBF_user.rename(columns={"Unnamed: 0": 'idx', "0": 'user_id'})

In [9]:
CBF_user

idx                 user_id
0          0  mh_-eMZ6K5RLWhZyISBhwA
1          1  ejFxLGqQcWNLdNByJlIhnQ
2          2  f7xa0p_1V9lx53iIGN5Sug
3          3  qdneUwCll9ADHOp7i3wq3Q
4          4  QD9xhB-261YIQIFI5sRBtQ
...      ...                     ...
25503  25503  Dxp82KxFcmisgeCxHomfVg
25504  25504  wGqF01YaUYyF0aK8R8H9Lw
25505  25505  roGoHGWTkYrerml4wr6Wmg
25506  25506  P_bWyqJNstfYrFSLQjGrrQ
25507  25507  pwJJW342YyjEyq1Mxc9wXw

[25508 rows x 2 columns]

In [10]:
CBF_user_map = {}
for idx, row in CBF_user.iterrows():
    CBF_user_map[row['user_id']] = row['idx']

In [11]:
with open('../CF_user_map', 'rb') as f:
    CF_user_map = pickle.load(f)

In [12]:
rest_list = pd.read_csv('../rest_list.csv', index_col=False)
rest_list = rest_list.rename(columns = {"Unnamed: 0": "idx", "0": "business_id"})

In [13]:
rest_list

idx             business_id
0        0  XQfwVwDr-v0ZS3_CbbE5Xw
1        1  otQS34_MymijPTdNBoBdCw
2        2  kq5Ghhh14r-eCxlVmlyd8w
3        3  18eWJFJbXyR9j_5xfcRLYA
4        4  W7NxQw8UYFR0HLPrI08tvw
...    ...                     ...
2451  2451  PjjShgqNvX5Marv-RJ1oNg
2452  2452  qQpgF5fGMTulUUtjzlsnwg
2453  2453  g0OWissnXQgRJVJGYv1mfQ
2454  2454  1liiTsfPbK_POsNp52A7NQ
2455  2455  5R3-eCIk4dRBtXo0A5MAzQ

[2456 rows x 2 columns]

In [14]:
CBF_busi_map = {}
for idx, row in rest_list.iterrows():
    CBF_busi_map[row['idx']] = row['business_id'] #encode: business_id

In [15]:
with open('../CF_business_map', 'rb') as f:
    CF_busi_map = pickle.load(f)  #encode: business_id

In [16]:
for user_id in CBF_user_map.keys():
    if user_id in CF_user_map.values():
        CBF_recom_user = CBF_recom[CBF_user_map[user_id]]
        idx_CF = list(CF_user_map.keys())[list(CF_user_map.values()).index(user_id)] #index in CF
        for i, li in enumerate(CF_user):
            if idx_CF in li:
                j = li.index(idx_CF)
                idx = (i, j)
        CF_recom_user = CF_recom[idx[0]][idx[1]]
        CBF_recom_user = [CBF_busi_map[i.item()] for i in CBF_recom_user]
        CBF_recom_user = [list(CF_busi_map.keys())[list(CF_busi_map.values()).index(i)] for i in CBF_recom_user]
        final = torch.concat([CF_recom_user[:10], torch.tensor(CBF_recom_user[:10])])
        CF_recom[idx[0]][idx[1]] = final

In [17]:
with open('CF_recom_hybrid_filtered', 'wb') as f:
    pickle.dump(CF_recom, f)